<a href="https://colab.research.google.com/github/amtayl25/1d3d/blob/main/3d1d_advdiff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Install required libraries (try-catch safe)
"""
NOTE: You can save the Docker image state after running this block so that you don't have to run it every time you start a new environment.
"""
import os, re

def replace_in_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Replace 'ufl' with 'ufl_legacy'
    content = re.sub(r'\bufl\b', 'ufl_legacy', content)

    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

def process_directory(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.py'):
                file_path = os.path.join(root, file)
                replace_in_file(file_path)

# ipywidgets
try:
    import ipywidgets
except ImportError:
    !pip install ipywidgets

# dolfin
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-release-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

# block
try:
    import block
except ImportError:
    !git clone "https://bitbucket.org/fenics-apps/cbc.block/src/master/"
    !pip install master/

# fenics_ii
try:
    import xii
except ImportError:
    !git clone "https://github.com/MiroK/fenics_ii"
    process_directory("fenics_ii/")
    !pip install fenics_ii/

# vtk
try:
    import vtk
except ImportError:
    !pip install vtk

# graphnics
try:
    import graphnics
except ImportError:
    !git clone "https://github.com/IngeborgGjerde/graphnics"
    !pip install graphnics/

# meshio
try:
    import meshio
except ImportError:
    !pip install meshio

# pyvista
try:
    import pyvista
except ImportError:
    !pip install pyvista

In [ ]:
# @title Convergence test: 3D-FEM,1D-dG time-dependent advection-diffusion

from math import gamma
from argparse import RawDescriptionHelpFormatter
from petsc4py import PETSc
from ast import Constant
from dolfin import *
from xii import *
import numpy as np
import time

# @title Testing

conv_study3d = open("conv_study3d.txt", "w")
conv_study1d = open("conv_study1d.txt", "w")

H1_error_3d   = np.ones(6)
H1_rates_3d   = np.ones(6)
L2_error_3d   = np.ones(6)
L2_rates_3d   = np.ones(6)
err_interface     = np.ones(6)
rates_interface   = np.ones(6)
h_out             = np.ones(6)

H1_error_1d = np.ones(6)
H1_rates_1d = np.ones(6)
L2_error_1d = np.ones(6)
L2_rates_1d = np.ones(6)

kappa     = 1.0
gamma_in  = 1.0
gamma_out = 1.0

mesh_vec = [0,1,2,3,4]

T    = 1.0

for i in mesh_vec:
  ncell   = 4*(2**i)
  mesh_3d = BoxMesh(Point(-0.5,-0.5,-0.5),Point(0.5,0.5,0.5),ncell,ncell,ncell)
  radius  = 0.05
  dt      = 0.1#/ncell
  inv_dt  = Constant(1.0/dt)
  tpts    = int(T/dt)

  # === Perimeter and Area ===
  D_perimeter = 2*np.pi*radius
  D_area = np.pi*(radius**2)

  # === Create Meshes ===
  Omega = MeshFunction("size_t",mesh_3d,1,0)
  CompiledSubDomain("near(x[0],0.0) && near(x[1],0.0)").mark(Omega,1)
  Lambda = EmbeddedMesh(Omega,1)
  Lambda_boundaries = MeshFunction("size_t", Lambda, Lambda.topology().dim() - 1, 0)
  CompiledSubDomain("near(x[2], -0.5)").mark(Lambda_boundaries, 1)
  CompiledSubDomain("near(x[2],  0.5)").mark(Lambda_boundaries, 2)

  # === Function Spaces ===
  V3 = FunctionSpace(mesh_3d, "CG", 1)
  V1 = FunctionSpace(Lambda, "DG", 1)
  W = [V3, V1]
  u3, u1 = map(TrialFunction, W)
  v3, v1 = map(TestFunction, W)

  # === Velocity terms ===
  q1 = Constant(1.0)
  q3 = as_vector((0,0,q1))

  # === Initial concentration ===
  cinlet = Expression("(sin(pi*x[2])+2)*t", degree=4, t=0.0)#Constant(1.0)


  # === Mesh Measures ===
  dxOmega = Measure("dx", domain=mesh_3d)
  dxLambda = Measure("dx", domain=Lambda)
  dsLambda = Measure("ds", domain=Lambda, subdomain_data=Lambda_boundaries)
  dSLambda = Measure("dS", domain=Lambda)
  n = FacetNormal(Lambda)
  h = CellDiameter(Lambda)
  alpha = Constant(50.0)
  epsilon = Constant(-1.0)
  perf1 = 1.0
  perf3 = 1.0

  # === Upwind ===
  u1_up = conditional(ge(q1, 0.0), u1('-'), u1('+'))  # upstream trace


  # === Cylinder ===
  cylinder = Circle(radius=radius, degree=20)

  # === Averages ===
  u3_avg = Average(u3, Lambda, cylinder)
  v3_avg = Average(v3, Lambda, cylinder)

  # === Initialize time step ===
  uh3d_prev = interpolate(Expression('0.0',degree=1), V3)
  uh1d_prev = interpolate(Expression('0.0',degree=1), V1)

  r = Expression('sqrt(pow(x[0] - 0.0, 2)+ pow(x[1] - 0.0,2))', degree  = 7)

  # === Begin time-stepping ===
  t = 0.0
  for step in range(tpts):
    t += dt
    print(f"[INFO] Time step {step+1}/{tpts}, t = {t:.3}")

    cinlet.t = t

    # === Exact solutions ===
    u3_exact_expr = Expression('(r > radius ? (kappa/(1+kappa))*(1-radius*std::log(r/radius))*((sin(pi*x[2])+2)*t) : (kappa/(1+kappa))*((sin(pi*x[2])+2)*t))',
                              degree = 7,
                              radius = radius,
                              kappa = kappa ,
                              r = r,
                              t = t)
    u3_exact = interpolate(u3_exact_expr, V3)

    u1_exact_expr = Expression('(sin(pi*x[2])+2)*t', degree = 7, t=t)
    u1_exact = interpolate(u1_exact_expr, V1)
    u1_inlet_expr    = Expression('(sin(pi*x[2])+2)*t', degree=7, t=t)  # right now same as u1_exact but can be changed in future for diff circumstance
    du1ds_exact_expr = Expression('pi*t*cos(pi*x[2])', degree=7, t=t)

    theta0 = Expression('area*(du - q*(u - uin))',
                    degree=7, area=D_area, q=q1,
                    du=du1ds_exact_expr, u=u1_exact_expr, uin=u1_inlet_expr)

    thetaN = Expression('area*du', degree=7, area=D_area, du=du1ds_exact_expr)

    vtkfile = File('u3_exact.pvd')
    vtkfile << u3_exact
    vtkfile = File('u1_exact.pvd')
    vtkfile << u1_exact

    # === Forcing terms ===
    f3 = Expression('(r > radius ? '
                    '((kappa/(1+kappa))*(1-radius*std::log(r/radius))*(sin(pi*x[2])+2) '             # time derivative
                    '+ (kappa/(1+kappa))*(1-radius*std::log(r/radius))*pow(pi,2)*t*sin(pi*x[2]) '    # diffusion
                    '+ (kappa/(1+kappa))*(1-radius*std::log(r/radius))*pi*t*q*cos(pi*x[2]) )'        # advection
                    ': ((kappa/(1+kappa))*pow(pi,2)*t*sin(pi*x[2]) '            # diffusion
                    '+ (kappa/(1+kappa))*(sin(pi*x[2])+2) '                     # time derivative
                    '+ (kappa/(1+kappa))*pi*t*q*cos(pi*x[2]) ) )',                  # advection
                    degree = 4,
                    kappa = kappa,
                    radius = radius,
                    area = D_area,
                    r = r,
                    t = t,
                    q = q1)

    f1 = Expression(
        'area * (sin(pi*x[2])+2)'                             # time derivative
        ' + area * pow(pi,2) * t * sin(pi*x[2])'               # diffusion
        ' + (kappa / (1 + kappa)) * perimeter * t * (sin(pi*x[2]) + 2)'  # coupling
        ' + area * pi * t * q * cos(pi*x[2])',                # advection
        degree=4,
        area=D_area,
        perimeter=D_perimeter,
        kappa=kappa,
        t=t,
        q=q1
    )

    # === Variational Forms ===
    a00 = inv_dt * (inner(u3, v3) * dxOmega) + inner(grad(u3), grad(v3)) * dxOmega \
          + D_perimeter * kappa * inner(u3_avg, v3_avg) * dxLambda - inner(q3, grad(v3)) * u3 * dxOmega
    a01 = -kappa * D_perimeter * inner(u1, v3_avg) * dxLambda
    a10 = -kappa * D_perimeter * inner(u3_avg, v1) * dxLambda
    a11 = D_area * inv_dt * u1 * v1 * dxLambda \
        + D_area * inner(grad(u1), grad(v1)) * dxLambda \
        - D_area * inner(avg(grad(u1)), jump(v1, n)) * dSLambda \
        - D_area * inner(avg(grad(v1)), jump(u1, n)) * dSLambda \
        + (alpha/avg(h)) * jump(u1) * jump(v1) * dSLambda \
        + kappa * D_perimeter * u1 * v1 * dxLambda \
        - D_area * q1 * u1 * v1.dx(2) * dxLambda \
        + D_area*q1*u1_up*jump(v1)*dSLambda + D_area * q1 * u1 * v1 * dsLambda(2)


    L0 = inner(f3, v3) * dxOmega + inv_dt * (inner(uh3d_prev, v3) * dxOmega)
    L1 = inner(f1, v1) * dxLambda \
        + D_area * inv_dt * uh1d_prev * v1 * dxLambda \
        + D_area * q1 * cinlet * v1 * dsLambda(1)  \
        - theta0*v1*dsLambda(1) + thetaN*v1*dsLambda(2)


    a = [[a00, a01], [a10, a11]]
    L = [L0, L1]

    bc3d = DirichletBC(V3, u3_exact, "on_boundary")
    W_bcs = [bc3d, []]  # No Dirichlet BCs on 1D

    A, b = map(ii_assemble, (a, L))
    A, b = apply_bc(A, b, W_bcs)
    A, b = map(ii_convert, (A, b))

    solver = PETScKrylovSolver()
    solver.set_operators(A, A)
    ksp = solver.ksp()

    opts = PETSc.Options()
    opts.setValue('ksp_type', 'gmres')
    opts.setValue('ksp_norm_type', 'unpreconditioned')
    opts.setValue('ksp_atol', 1E-14)
    opts.setValue('ksp_rtol', 1E-30)
    opts.setValue('ksp_monitor_true_residual', None)
    opts.setValue('pc_type', 'hypre')
    ksp.setFromOptions()

    x = b.copy()
    solver.solve(A,x,b)

    wh = ii_Function(W)
    wh.vector()[:] = x
    uh3d, uh1d = wh

    uh3d_prev.assign(uh3d)
    uh1d_prev.assign(uh1d)

    avg_uh3d = Average(uh3d, Lambda, cylinder)
    avg_u3d_exact = Average(u3_exact, Lambda, cylinder)

    # === Store Model Predictions ===
    vtkfile = File('3d_Prediction.pvd')
    vtkfile << uh3d

    vtkfile = File('1d_Prediction.pvd')
    vtkfile << uh1d


  # === Computer Errors ===
  h_out[i] =uh3d.function_space().mesh().hmax()
  H1_error_3d[i] = errornorm(u3_exact_expr, uh3d, 'H1')
  L2_error_3d[i] = errornorm(u3_exact_expr, uh3d, 'L2')
  err_interface[i] = 1.0
  H1_rates_3d[i] = ln(H1_error_3d[i]/H1_error_3d[i-1])/ln(h_out[i]/h_out[i-1])
  L2_rates_3d[i] = ln(L2_error_3d[i]/L2_error_3d[i-1])/ln(h_out[i]/h_out[i-1])
  rates_interface[i] = ln(err_interface[i]/err_interface[i-1])/ln(h_out[i]/h_out[i-1])

  H1_error_1d[i] = errornorm(u1_exact_expr, uh1d, 'H1')
  L2_error_1d[i] = errornorm(u1_exact_expr, uh1d, 'L2')
  H1_rates_1d[i] = ln(H1_error_1d[i]/H1_error_1d[i-1])/ln(h_out[i]/h_out[i-1])
  L2_rates_1d[i] = ln(L2_error_1d[i]/L2_error_1d[i-1])/ln(h_out[i]/h_out[i-1])

  print(i,"{:.3e}".format(h_out[i]),"{:.3e}".format(H1_error_3d[i]), "{:.3e}".format(H1_rates_3d[i]), "{:.3e}".format(err_interface[i]),"{:.3e}".format(rates_interface[i]), "{:.3e}".format(L2_error_3d[i]),"{:.3e}".format(L2_rates_3d[i]),  file= conv_study3d,flush = True)
  print(i,"{:.3e}".format(ncell),"{:.3e}".format(H1_error_1d[i]), "{:.3e}".format(H1_rates_1d[i]), "{:.3e}".format(L2_error_1d[i]),"{:.3e}".format(L2_rates_1d[i]),  file= conv_study1d,flush = True)


[INFO] Time step 1/10, t = 0.1


Averaging over 4 cells: 100%|██████████| 4/4 [00:00<00:00, 357.43it/s]

  0 KSP unpreconditioned resid norm 1.005605394005e+00 true resid norm 1.005605394005e+00 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 3.292179726944e-04 true resid norm 3.292179726902e-04 ||r(i)||/||b|| 3.273828627538e-04
  2 KSP unpreconditioned resid norm 1.363379425031e-06 true resid norm 1.363379423433e-06 ||r(i)||/||b|| 1.355779743784e-06
  3 KSP unpreconditioned resid norm 3.790447539773e-10 true resid norm 3.790457514908e-10 ||r(i)||/||b|| 3.769328941057e-10
  4 KSP unpreconditioned resid norm 8.306159822007e-14 true resid norm 8.130084639104e-14 ||r(i)||/||b|| 8.084766338336e-14
  5 KSP unpreconditioned resid norm 3.249897033146e-16 true resid norm 1.442626344164e-14 ||r(i)||/||b|| 1.434584930395e-14
[INFO] Time step 2/10, t = 0.2
  0 KSP unpreconditioned resid norm 2.010464787039e+00 true resid norm 2.010464787039e+00 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 6.592239883693e-04 true resid norm 6.592239883596e-04 ||r(i)||/||

  0 KSP unpreconditioned resid norm 5.030816484868e+00 true resid norm 5.030816484868e+00 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 1.653622225428e-03 true resid norm 1.653622225405e-03 ||r(i)||/||b|| 3.286985781293e-04
  2 KSP unpreconditioned resid norm 6.846453027001e-06 true resid norm 6.846453018002e-06 ||r(i)||/||b|| 1.360902954539e-06
  3 KSP unpreconditioned resid norm 1.912821582840e-09 true resid norm 1.912895024413e-09 ||r(i)||/||b|| 3.802355005727e-10
  4 KSP unpreconditioned resid norm 4.196665502449e-13 true resid norm 4.403573772370e-13 ||r(i)||/||b|| 8.753198979957e-14
  5 KSP unpreconditioned resid norm 5.754731493462e-16 true resid norm 8.630593106586e-14 ||r(i)||/||b|| 1.715545206736e-14
[INFO] Time step 6/10, t = 0.6
  0 KSP unpreconditioned resid norm 6.039840174681e+00 true resid norm 6.039840174681e+00 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 1.986983905738e-03 true resid norm 1.986983905745e-03 ||r(i)||/||

Averaging over 8 cells: 100%|██████████| 8/8 [00:00<00:00, 256.64it/s]

...

  0 KSP unpreconditioned resid norm 3.807284709187e+00 true resid norm 3.807284709187e+00 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 4.196430950852e-03 true resid norm 4.196430950843e-03 ||r(i)||/||b|| 1.102210964343e-03
  2 KSP unpreconditioned resid norm 1.860029583526e-05 true resid norm 1.860029583587e-05 ||r(i)||/||b|| 4.885449147260e-06
  3 KSP unpreconditioned resid norm 4.996230610420e-08 true resid norm 4.996228478739e-08 ||r(i)||/||b|| 1.312281287156e-08
  4 KSP unpreconditioned resid norm 4.237706833772e-10 true resid norm 4.237237841261e-10 ||r(i)||/||b|| 1.112929072795e-10
  5 KSP unpreconditioned resid norm 1.085564720827e-12 true resid norm 1.106855179379e-12 ||r(i)||/||b|| 2.907203595013e-13
  6 KSP unpreconditioned resid norm 3.325323296321e-15 true resid norm 8.630084151723e-14 ||r(i)||/||b|| 2.266729391395e-14
[INFO] Time step 3/10, t = 0.3
  0 KSP unpreconditioned resid norm 5.710972678118e+00 true resid norm 5.710972678118e+00 ||r(i)||/||

Averaging over 16 cells: 100%|██████████| 16/16 [00:00<00:00, 411.99it/s]


  0 KSP unpreconditioned resid norm 3.767167014479e+00 true resid norm 3.767167014479e+00 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 2.173192790422e-03 true resid norm 2.173192790427e-03 ||r(i)||/||b|| 5.768772082772e-04
  2 KSP unpreconditioned resid norm 2.395461468667e-05 true resid norm 2.395461469256e-05 ||r(i)||/||b|| 6.358787545254e-06
  3 KSP unpreconditioned resid norm 2.380153498460e-07 true resid norm 2.380153368369e-07 ||r(i)||/||b|| 6.318151967304e-08
  4 KSP unpreconditioned resid norm 2.939841048497e-09 true resid norm 2.939819377859e-09 ||r(i)||/||b|| 7.803793584304e-10
  5 KSP unpreconditioned resid norm 3.927925655140e-11 true resid norm 3.925597867283e-11 ||r(i)||/||b|| 1.042055701856e-11
  6 KSP unpreconditioned resid norm 4.153156171801e-13 true resid norm 4.312860597598e-13 ||r(i)||/||b|| 1.144855160661e-13
  7 KSP unpreconditioned resid norm 5.294549440177e-15 true resid norm 1.467057403858e-13 ||r(i)||/||b|| 3.894325359666e-14
[INFO] T

Averaging over 32 cells: 100%|██████████| 32/32 [00:00<00:00, 371.28it/s]


  0 KSP unpreconditioned resid norm 7.517646353441e+00 true resid norm 7.517646353441e+00 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 2.076017632531e-03 true resid norm 2.076017632523e-03 ||r(i)||/||b|| 2.761526061375e-04
  2 KSP unpreconditioned resid norm 2.914993423317e-05 true resid norm 2.914993422444e-05 ||r(i)||/||b|| 3.877534650336e-06
  3 KSP unpreconditioned resid norm 3.546873454181e-07 true resid norm 3.546873214117e-07 ||r(i)||/||b|| 4.718063403572e-08
  4 KSP unpreconditioned resid norm 4.833553782680e-09 true resid norm 4.833532396587e-09 ||r(i)||/||b|| 6.429582038499e-10
  5 KSP unpreconditioned resid norm 6.005613069021e-11 true resid norm 6.006009777582e-11 ||r(i)||/||b|| 7.989215633737e-12
  6 KSP unpreconditioned resid norm 6.940477074699e-13 true resid norm 8.177115327347e-13 ||r(i)||/||b|| 1.087722798187e-13
  7 KSP unpreconditioned resid norm 9.865959820637e-15 true resid norm 3.958449345038e-13 ||r(i)||/||b|| 5.265543441301e-14
[INFO] T